In [16]:
import json
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, MaxPooling1D

from mediapipe_impl.hand_track.HandTrackingMin import success


In [79]:
def load_data(json_path):
    with open(json_path, 'r') as f:
        data = json.load(f)

    inputs = []
    labels = []
    for sample in data:
        features = sample["features"]
        # Flatten each (x, y, z, visibility) into a single array
        flattened_features = []
        for keypoint in features:
            flattened_features.extend([keypoint["x"], keypoint["y"], keypoint["z"], keypoint["visibility"]])

        inputs.append(flattened_features)

        # Multi-label processing: Convert label list to a binary vector
        label_list = sample["label"]
        labels.append(label_list)

    return np.array(inputs), labels

In [87]:
# Load data
X, y = load_data('../data.json')
print(f"Initial X shape: {X.shape}")
ALL_LABELS = []
for label in y:
    if label not in ALL_LABELS:
        ALL_LABELS.append(label)
print(f"ALL LABELS: {ALL_LABELS}")
# N categories
N = len(ALL_LABELS)
print(f"N: {N}")
KEY_POINTS = 33
FEATURES = 4
# Reshape data to fit Conv1D input: (samples, steps, features)
X = X.reshape((X.shape[0], KEY_POINTS, 4))  # 33 keypoints with 4 features (x, y, z, visibility)
# Checking the shape of the reshaped data
print(X.shape)

Initial X shape: (2, 132)
ALL LABELS: ['world', 'hello']
N: 2
(2, 33, 4)


In [88]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

# Initialize a LabelEncoder to convert strings to integers
label_encoder = LabelEncoder()
# Fit and transform the labels to integers
y_int = label_encoder.fit_transform(y)
# Now apply to_categorical for one-hot encoding
y_onehot = to_categorical(y_int, num_classes=len(label_encoder.classes_))
print(f"One-hot encoded labels shape: {y_onehot.shape}")
print(y_onehot)


One-hot encoded labels shape: (2, 2)
[[0. 1.]
 [1. 0.]]


In [89]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout

# Define the model
model = Sequential()
# Add Conv1D layer
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(KEY_POINTS, FEATURES)))
# Add MaxPooling1D layer
model.add(MaxPooling1D(pool_size=2))
# Add another Conv1D layer
model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
# Add another MaxPooling1D layer
model.add(MaxPooling1D(pool_size=2))
# Flatten the output from Conv1D layers
model.add(Flatten())
# Add Dense layer with dropout for regularization
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
# Output layer with softmax activation (for classification)
model.add(Dense(N, activation='softmax'))  # N is the number of classes

/Users/mikeshinoda/Library/Caches/pypoetry/virtualenvs/advanced-computer-vision-mRP4g53R-py3.12/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [90]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# Summary of the model
model.summary()

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_14 (Conv1D)              │ (None, 31, 64)         │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_14 (MaxPooling1D) │ (None, 15, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_15 (Conv1D)              │ (None, 13, 128)        │        24,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_15 (MaxPooling1D) │ (None, 6, 128)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_7 (Flatten)             │ (None, 768)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 128)            │        98,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 2)              │           258 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 124,226 (485.26 KB)

 Trainable params: 124,226 (485.26 KB)

 Non-trainable params: 0 (0.00 B)

In [70]:
# Train the Model
history = model.fit(X, y_onehot, epochs=10, batch_size=32, validation_split=0.2)
# Evaluate the Model
loss, accuracy = model.evaluate(X, y_onehot)
print(f'Accuracy: {accuracy * 100:.2f}%')

Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - accuracy: 1.0000 - loss: 0.0286 - val_accuracy: 0.0000e+00 - val_loss: 4.4675
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 1.0000 - loss: 0.0158 - val_accuracy: 0.0000e+00 - val_loss: 4.9717
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 1.0000 - loss: 0.0144 - val_accuracy: 0.0000e+00 - val_loss: 5.4782
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 1.0000 - loss: 0.0024 - val_accuracy: 0.0000e+00 - val_loss: 5.9735
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 1.0000 - loss: 0.0031 - val_accuracy: 0.0000e+00 - val_loss: 6.4554
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 1.0000 - loss: 3.2765e-04 - val_accuracy: 0.0000e+00 - val_loss: 6.9152
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 1.0000 - loss: 0.0053 - val_accuracy: 0.0000e+00 - val_loss: 7.3631
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 1.0000 - loss: 0.0010 - val_accu

In [71]:
import cv2
from mediapipe_impl.pose_estimation import PoseEstimationModule as pm

detector = pm.PoseDetector()


def extract_keypoints(image_path):
    cap = cv2.VideoCapture(image_path)
    success, img = cap.read()
    img = detector.find_pose(img=img)
    lm_list = detector.find_position(img, draw=False)
    return lm_list

I0000 00:00:1731676641.330146 4763801 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 83.1), renderer: Apple M1


In [85]:
def predict_image(image_path):
    data = []
    keypoints = extract_keypoints(image_path)
    for keypoint in keypoints:
        data.extend([keypoint["x"], keypoint["y"], keypoint["z"], keypoint["visibility"]])
    X = np.array(data).reshape((1, 33, 4))
    print(X.shape)
    # 模型预测
    predictions = model.predict(X)

    # 获取预测类别的索引
    predicted_class = np.argmax(predictions, axis=1)[0]

    print(f"Predicted Class: {predicted_class}")
    print(ALL_LABELS[predicted_class])


# 示例：预测一张新图像
image_path = '../datasets/img.jpg'
result = predict_image(image_path)

(1, 33, 4)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
Predicted Class: 1
hello
